# Air Pollution

## What is it?

**Air pollution** refers to the release of pollutants into the air that are detrimental to human health and the planet as a whole. It can be described as one of the most dangerous threats that the humanity ever faced. It causes damage to animals, crops, forests, and  wate-bodiesr. It also contributes to the depletion of the ozone layer, which protects the Earth from the sun's UV rays. Some of the other environmental effects of air pollution are haze, eutrophication, and global climate changes.

## What causes Air Pollution?

Most air pollution comes from **energy use and production**. **Burning fossil fuels** releases gases and chemicals into the air. And in an especially destructive feedback loop, air pollution not only contributes to climate change but is also exacerbated by it. Air pollution in the form of carbon dioxide and methane raises the earth’s temperature. Another type of air pollution is then worsened by that increased heat: **Smog** forms when the weather is warmer and there’s more **ultraviolet radiation**. **Climate change** also increases the production of allergenic air pollutants including mold (thanks to damp conditions caused by extreme weather and increased flooding) and pollen (due to a longer pollen season and more pollen production).


# About the notebook

I am trying to gain expertize in Data Science field and I just completed the **SQL Scavenger Hunt** tutorial where I came to know about BigQuery. I had fun in learning and exploring the tutorial and trying my fingers on the code. So, here I am to explore my first ever kernel on BigQuery.

In this notebook data is extracted from BigQuery Public Data assesible exclusively only in Kaggle. The BigQurey Helper Object will convert data in cloud storage into Pandas DataFrame object. The query syntax is same as SQL. As size of data is very high convert entire data to DataFrame is cumbersome. So query is written such that will be readly available for Visualization.

> **BigQuery** is a RESTful web service that enables interactive analysis of massively large datasets working in conjunction with Google Storage. It is an Infrastructure as a Service that may be used complementarily with MapReduce.

Here, I will be analyzing OpenAQ dataset, which has some information about the air pollutants. 

**Measurement units**
       a.  ug/m3 : micro gram/cubic meter
       b.  ppm : Parts Per Million

**Pollutants**
    a. O3 : Ozone gas
    b. SO2 : Sulphur Dioxed
    c. NO2 : Nitrogen Dioxed
    d. PM2.5 : Particles with an aerodynamic diameter less than  2.5μm 
    e. PM10 : Particles with an aerodynamic diameter less than  10μm 
    f. CO : Carbon monoxide

So, let's dive in immediately and find out the results!

# Importing the necessary libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows =10

import bq_helper

# google bigquery library for quering data
from google.cloud import bigquery

# BigQueryHelper for converting query result direct to dataframe
from bq_helper import BigQueryHelper

# Importing Libraries for Plotting
## (For Data Visualization)

In [ ]:
# matplotlib for plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

# import plotly
import plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as fig_fact
plotly.tools.set_config_file(world_readable=True, sharing='public')

from mpl_toolkits.basemap import Basemap
import folium
import folium.plugins as plugins

%matplotlib inline

# Starting with the dataset

Now, we will be creating a dataset and list down all the tables for this particular dataset.

In [ ]:
# create a helper object for this dataset
open_aq = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="openaq")


# print all the tables in this dataset (there's only one!)
open_aq.list_tables()

In [ ]:
#Schema 
open_aq.table_schema('global_air_quality')

So, we can see that we have only one table (**global_air_quality**) in the dataset **openaq**.

Let's find out what are the different attributes for the table.

### Display Table *global_air_quality*

In [ ]:
# print the first couple rows of the "global_air_quality" dataset
open_aq.head("global_air_quality")

# Air Quality Index(AQI) : Over the Years

In [ ]:
query_aqi = """
            SELECT EXTRACT(YEAR FROM timestamp) as `Year`,
                   AVG(value) as `Average`,
                   latitude,
                   longitude
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE unit = 'µg/m³'
        GROUP BY Year, 
                 latitude,
                 longitude
        """
aqi = open_aq.query_to_pandas_safe(query_aqi)

aqi.head()

In [ ]:
from matplotlib import animation,rc
import io
import base64
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')
fig = plt.figure(figsize=(14,10))
plt.style.use('ggplot')

def animate(Year):
    ax = plt.axes()
    ax.clear()
    ax.set_title('AQI in Year: '+str(Year))
    m4 = Basemap(llcrnrlat=-90, urcrnrlat=90, llcrnrlon=-180,urcrnrlon=180,projection='cyl')
    m4.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
    m4.fillcontinents(color='grey', alpha=0.3)
    m4.drawcoastlines(linewidth=0.1, color="white")
    m4.shadedrelief()
    
    lat_y = list(aqi[aqi['Year'] == Year]['latitude'])
    lon_y = list(aqi[aqi['Year'] == Year]['longitude'])
    lat,lon = m4(lat_y,lon_y) 
    avg = np.log(aqi[aqi['Year'] == Year]['Average'])
    m4.scatter(lon,lat,c = avg,lw=2, alpha=0.3,cmap='hot_r')
   
ani = animation.FuncAnimation(fig,animate,list(aqi['Year'].unique()), interval = 1500)    
ani.save('animation.gif', writer='imagemagick', fps=1)
plt.close(1)
filename = 'animation.gif'
video = io.open(filename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<img src="data:image/gif;base64,{0}" type="gif" />'''.format(encoded.decode('ascii')))

We can see from the above animated image that the number of countries affected by pollutants increases over the number of years.
Now, we will have a find out the  countries listed in the AQI.

# Countries listed in Air Quality Index (AQI)

In [ ]:
query = """SELECT country,COUNT(country) as `count`
    FROM `bigquery-public-data.openaq.global_air_quality`
    GROUP BY country
    HAVING COUNT(country) >10
    ORDER BY `count` DESC
    """
cnt = open_aq.query_to_pandas_safe(query)

cnt.head()

In [ ]:
plt.style.use('bmh')
plt.figure(figsize=(15,5))
sns.barplot(cnt['country'], cnt['count'], palette='magma')
plt.xticks(rotation=45)
plt.title('Distribution of country listed in data');

# Locations

Shown below are some locations where average AQI reports to be the maximum.

Other locations in World Map are also shown below.

In [ ]:
query = """SELECT city,latitude,longitude,averaged_over_in_hours,
            AVG(value) as `Average`
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE unit = 'µg/m³'
            GROUP BY latitude,city,longitude,averaged_over_in_hours   
            ORDER BY Average DESC
            """
location = open_aq.query_to_pandas_safe(query)
location.dropna(axis=0, inplace=True)
location.head(10)

We can see that Indian cities are holding the topmost places in the leaderboard.
But, there are many other places in the dataset which are also not lagging.
We can see those places in the map below.

# AQI Value Distribuution Map View

In [ ]:
plt.style.use('ggplot')
f,ax = plt.subplots(figsize=(15,10))
m1 = Basemap(projection='cyl', llcrnrlon=-180, urcrnrlon=180, llcrnrlat=-90, urcrnrlat=90,
            resolution='c',lat_ts=True)

m1.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m1.fillcontinents(color='grey', alpha=0.3)
m1.drawcoastlines(linewidth=0.1, color="white")
m1.shadedrelief()

hour = location['averaged_over_in_hours']
avg = np.log(location['Average'])
m1loc = m1(location['latitude'].tolist(),location['longitude'])
m1.scatter(m1loc[1],m1loc[0],lw=3,alpha=0.5,cmap='hot_r',\
          c=avg,s=hour)
plt.title('Average Air qulity index in unit $ug/m^3$ value')
plt.colorbar(label=' Average Log AQI value in unit $ug/m^3$');

So, we can see that the pollution level in different places are at an alarming condition. The European countries(topping the list, USA,South and South-East Asian conutries are making more pollution, where as Australia, Africa and South Africa do not make up to the topper's list.

# Countries producing more pollution
## Digging Deeper

In [ ]:
#USA location
query = """SELECT 
            MAX(latitude) as `max_lat`,
            MIN(latitude) as `min_lat`,
            MAX(longitude) as `max_lon`,
            MIN(longitude) as `min_lon`
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'US' 
        """
us_loc = open_aq.query_to_pandas_safe(query)
us_loc

In [ ]:
query = """ SELECT city,latitude,longitude,averaged_over_in_hours,
            AVG(value) as `Average`
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'US' AND unit = 'µg/m³'
            GROUP BY latitude,city,longitude,averaged_over_in_hours,country
            ORDER BY Average DESC
        """

us_aqi = open_aq.query_to_pandas_safe(query)
us_aqi.head()

In [ ]:
# USA
min_lat = us_loc['min_lat']
max_lat = us_loc['max_lat']
min_lon = us_loc['min_lon']
max_lon = us_loc['max_lon']

plt.figure(figsize=(14,10))
m2 = Basemap(projection='cyl', llcrnrlon=min_lon, urcrnrlon=max_lon, llcrnrlat=min_lat, urcrnrlat=max_lat,
            resolution='c',lat_ts=True)
m2.drawcounties()
m2.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m2.fillcontinents(color='grey', alpha=0.3)
m2.drawcoastlines(linewidth=0.1, color="white")
m2.drawstates()
avg = np.log((us_aqi['Average']))
h = us_aqi['averaged_over_in_hours']
m2loc = m2(us_aqi['latitude'].tolist(),us_aqi['longitude'])
m2.scatter(m2loc[1],m2loc[0],s = h,c = avg,lw=3,alpha=0.2,zorder=3,cmap='hot_r')
plt.colorbar(label = 'Average Log AQI value in unit $ug/m^3$')
plt.title('Average Air qulity index in unit $ug/m^3$ of US')

In [ ]:
#INDIA location
query = """SELECT 
            MAX(latitude) as `max_lat`,
            MIN(latitude) as `min_lat`,
            MAX(longitude) as `max_lon`,
            MIN(longitude) as `min_lon`
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'IN' """
in_loc = open_aq.query_to_pandas_safe(query)
in_loc

In [ ]:
query = """ SELECT city,latitude,longitude,averaged_over_in_hours,
            AVG(value) as `Average`
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE country = 'IN' AND unit = 'µg/m³'
            GROUP BY latitude,city,longitude,averaged_over_in_hours,country """
in_aqi = open_aq.query_to_pandas_safe(query)
in_aqi

In [ ]:
# INDIA
min_lat = in_loc['min_lat']
max_lat = in_loc['max_lat']
min_lon = in_loc['min_lon']
max_lon = in_loc['max_lon']

plt.figure(figsize=(15,7))
m3 = Basemap(projection='cyl', llcrnrlon=min_lon, urcrnrlon=max_lon, llcrnrlat=min_lat, urcrnrlat=max_lat,
            resolution='c',lat_ts=True)
m3.drawcounties()
m3.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m3.fillcontinents(color='grey', alpha=0.3)
m3.drawcoastlines(linewidth=0.1, color="white")
m3.drawstates()
avg = np.log((in_aqi['Average']))
h = in_aqi['averaged_over_in_hours']
m3loc = m3(in_aqi['latitude'].tolist(),in_aqi['longitude'])
m3.scatter(m3loc[1],m3loc[0],s = h,c = avg,lw=3,alpha=1,cmap='hot_r')
plt.colorbar(label = 'Average Log AQI value in unit $ug/m^3$')
plt.title('Average Air qulity index in unit $ug/m^3$ of India');

# Country Source

In the above dataset, we can see that there is an attribute called 'source_name'. This attribute points to the source (i.e. magazine/newspaper/media) which provides the pollution data for particular countries.
So, our next step would be to find out the different source names and their corresponding reporting countries. 

In [ ]:
query_source = """
                    SELECT 
                    DISTINCT source_name, 
                             country
                    FROM `bigquery-public-data.openaq.global_air_quality`
                    ORDER BY source_name
                    """

source = open_aq.query_to_pandas_safe(query_source)

source.head(10)

So, listed above are some of the countries along with their source providers. 
For example, AirNow provides **Air Quality Reports **for countries like Australia (AU), United States (US), Chile (CL), Mexico (MX) and Canada (CA), to be named a few. 

Now, we will be checking country count per source in the below query.

In [ ]:
query_country_count_per_source = """
                    SELECT 
                    DISTINCT source_name,
                    COUNT(country) AS Country_Count
                    FROM `bigquery-public-data.openaq.global_air_quality`
                    GROUP BY source_name
                    HAVING Country_Count > 50
                    ORDER BY Country_Count DESC
                    """
# Country_Count > 50 (Ignoring the negligible values)
country_count_per_source = open_aq.query_to_pandas_safe(query_country_count_per_source)

country_count_per_source.head(10)

According to the data produced above, we will now plot a graph to find out the leading sources(in terms of number of countries) to provide the **Air Quality Data/Pollution Report**.

# Graph Plot

In [ ]:
plt.subplots(figsize=(12,10))
sns.barplot(x='Country_Count',y='source_name',data=country_count_per_source,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Source Name', fontsize=15)
plt.xticks(rotation=45,fontsize=10)
plt.xlabel('Country Count', fontsize=15)
plt.title('Sources per Country Count', fontsize=24)
plt.savefig('sources_per_country_count.png')
plt.show()

Here, we can see that sources like **AirNow**, **EEA France**, **EEA Spain**, **EEA Germany**, **EEA Austria**, **EEA Czech Republic** tops the leaderboard.

So, for our further analysis, we will consider the pollutant data from the above stated sources.

Hence, the next step will be to identify the different pollutants.

# Different Pollutants

In [ ]:
query_pollutants = """
                    SELECT 
                    DISTINCT pollutant, unit
                    FROM `bigquery-public-data.openaq.global_air_quality`
                    WHERE source_name IN ('AirNow', 'EEA France', 'EEA Spain', 'EEA Germany', 'EEA Austria', 'EEA Czech Republic')
                    ORDER BY pollutant
                    """

pollutants = open_aq.query_to_pandas_safe(query_pollutants)

pollutants.head(15)

So, the noted pollutants are:

*  Black Carbon
*  Carbon Monoxide
*  Nitrogen Dioxide
*  Ozone
*  Sulphur Dioxide
*  pm10
*  pm25

Hence, we will be focusing more on these pollutants now.

# Black Carbon 

**Black carbon** is the **sooty black material **emitted from gas and diesel engines, coal-fired power plants, and other sources that burn fossil fuel. It comprises a significant portion of particulate matter or PM, which is an air pollutant.

**Black Carbon (BC) **has recently emerged as a major contributor to global climate change, possibly second only to CO2 as the main driver of change. BC particles strongly absorb sunlight and give soot its black color. BC is produced both naturally and by human activities as a result of the incomplete combustion of fossil fuels, biofuels, and biomass. Primary sources include emissions from diesel engines, cook stoves, wood burning and forest fires. Reducing CO2 emissions is essential to avert the worst impacts of future climate change, but CO2 has such a long atmospheric lifetime that it will take several decades for CO2 concentrations to begin to stabilize after emissions reductions begin. In contrast, BC remains in the atmosphere for only a few weeks, so cutting its emissions would immediately reduce the rate of warming, particularly in the rapidly changing Arctic. Moreover, reduced exposure to BC provides public health co-benefits, especially in developing countries. Technologies that can reduce global BC emissions are available today.

# Analysis
## (µg/m³)

In [ ]:
query_bc = """
    SELECT country, 
           source_name,
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'bc'
      AND unit = 'µg/m³'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

bc = open_aq.query_to_pandas_safe(query_bc)

bc.head(10)

As per **AirNow**(as mentioned in the dataset), **US** is the only country which produces **Black Carbon** with an average value of **1.4525**.
Hence, no graph plot could be produced.

# Carbon Monoxide 

**Carbon monoxide**, also known as CO, is called the "**Invisible Killer**" because it's a colorless, odorless, poisonous gas. More than 150 people in the Unites States die every year(not only in US, but also in other countries) from accidental non-fire related CO poisoning associated with consumer products, including generators. Other products include faulty, improperly-used or incorrectly-vented fuel-burning appliances such as furnaces, stoves, water heaters and fireplaces.

# Analysis - I
## (µg/m³)

In [ ]:
query_co = """
    SELECT country, 
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'co'
      AND unit = 'µg/m³'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

co = open_aq.query_to_pandas_safe(query_co)

co.head(10)

The query produced some country names above which has CO gas values (in µg/m³), and the graphical representation is shown below:

# Graph Plot

In [ ]:
plt.subplots(figsize=(15,6))
sns.barplot(x='country',y='Avg_Value',data=co,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Carbon Monoxide Gas values in µg/m³', fontsize=20)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('Country', fontsize=20)
plt.title('Average value of Carbon Monoxide gas in different countries', fontsize=24)
plt.savefig('avg_co.png')
plt.show()

The above plot shows Italy(IT) has the most negative gas values in µg/m³ (which indicates less pollution), whereas Mongolia (MN) tops the list. Countries like Lithuania (LI), Ireland (IE), Latvia (LV) and Belgium (BE) is on the verge of 0.

# Carbon Monoxide : Analysis - II
## (ppm)

In [ ]:
query_co_ppm = """
    SELECT country, 
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'co'
      AND unit = 'ppm'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

co_ppm = open_aq.query_to_pandas_safe(query_co_ppm)

co_ppm.head(10)

The query produced some country names above which has CO gas values (in ppm), and the graphical representation is shown below:

# Graph Plot

In [ ]:
plt.subplots(figsize=(15,7))
sns.barplot(x='country',y='Avg_Value',data=co_ppm,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Carbon Monoxide Gas values in ppm', fontsize=20)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('Country', fontsize=20)
plt.title('Average value of Carbon Monoxide gas in different countries', fontsize=24)
plt.savefig('avg_co_ppm.png')
plt.show()

The above plot shows Australia (AU) has less gas values in ppm, almost on the verge of 0 (which indicates less pollution), whereas Thailand (TH) tops the list. 

# Nitrogen Dioxide : Analysis I
## (µg/m³)

In [ ]:
query_no2 = """
    SELECT country, 
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'no2'
      AND unit = 'µg/m³'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

no2 = open_aq.query_to_pandas_safe(query_no2)

no2.head(5)

# Graph Plot

In [ ]:
plt.subplots(figsize=(15,6))
sns.barplot(x='country',y='Avg_Value',data=no2,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Nitrogen Dioxide Gas values in µg/m³', fontsize=20)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('Country', fontsize=20)
plt.title('Average value of Nitrogen Dinoxide gas in different countries', fontsize=24)
plt.savefig('avg_no2.png')
plt.show()

# Nitrogen Dioxide : Analysis II
## (ppm)

In [ ]:
query_no2_ppm = """
    SELECT country, 
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'no2'
      AND unit = 'ppm'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

no2_ppm = open_aq.query_to_pandas_safe(query_no2_ppm)

no2_ppm.head(5)

# Graph Plot

In [ ]:
plt.subplots(figsize=(15,6))
sns.barplot(x='country',y='Avg_Value',data=no2_ppm,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Nitrogen Dioxide Gas values in ppm', fontsize=20)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('Country', fontsize=20)
plt.title('Average value of Nitrogen Dinoxide gas in different countries', fontsize=24)
plt.savefig('avg_no2_ppm.png')
plt.show()

# Ozone : Analysis I
## (µg/m³)

In [ ]:
query_o3 = """
    SELECT country, 
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'o3'
      AND unit = 'µg/m³'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

o3 = open_aq.query_to_pandas_safe(query_o3)

o3.head(10)

# Graph Plot

In [ ]:
plt.subplots(figsize=(15,6))
sns.barplot(x='country',y='Avg_Value',data=o3,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Ozone Gas values in µg/m³', fontsize=20)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('Country', fontsize=20)
plt.title('Average value of Ozone gas in different countries', fontsize=24)
plt.savefig('avg_o3.png')
plt.show()

# Ozone : Analysis II
## (ppm)

In [ ]:
query_o3_ppm = """
    SELECT country, 
           avg(value) as Avg_Value
    FROM
      `bigquery-public-data.openaq.global_air_quality`
    WHERE
      pollutant = 'o3'
      AND unit = 'ppm'
      GROUP BY country, source_name
      ORDER BY Avg_Value ASC
        """

o3_ppm = open_aq.query_to_pandas_safe(query_o3_ppm)

o3_ppm.head(5)

# Graph Plot

In [ ]:
plt.subplots(figsize=(15,6))
sns.barplot(x='country',y='Avg_Value',data=o3_ppm,palette='inferno',edgecolor=sns.color_palette('dark',7))
plt.ylabel('Ozone Gas values in ppm', fontsize=20)
plt.xticks(rotation=90,fontsize=20)
plt.xlabel('Country', fontsize=20)
plt.title('Average value of Ozone gas in different countries', fontsize=24)
plt.savefig('avg_o3_ppm.png')
plt.show()

To be continued ......

I will be back soon with some more interesting visualizations. 
Till then, your valuable comments and critics are most welcome.
Please **upvote** if you appreciate my effort in the Data Science path.